In [2]:
import pandas as pd
import os

df = pd.read_csv(os.path.join(os.path.join(os.path.realpath('..'), "outputdata"), "results.csv"))

print(len(df))
# df = df[df.similarity_score != 1.0]
# print(len(df))
# df.to_csv(os.path.join(os.path.join(os.path.realpath('..'), "outputdata"), "results_cleaned.csv"),index=False)

219480


In [3]:
df.drop_duplicates(subset=None, keep='first', inplace=False)
overlapsDf = df[df["citation_link"] == True]
df = df.drop(['source_case','similar_case','source_case_topic','similarity_score'], axis='columns')
df = df[df["citation_link"] == True]
df = df.groupby(['method','citation_link']).citation_link.agg('count').to_frame('counts').reset_index()
df = df.sort_values('counts',ascending=False)
print(df.head(100))
print(len(df))

                          method  citation_link  counts
53        tfidf-quintgram-nostem           True     423
59          tfidf-sesgram-nostem           True     421
58                 tfidf-sesgram           True     421
52               tfidf-quintgram           True     420
57          tfidf-sepgram-nostem           True     417
56                 tfidf-sepgram           True     413
51         tfidf-quadgram-nostem           True     413
50                tfidf-quadgram           True     411
61          tfidf-trigram-nostem           True     402
55  tfidf-quintgram-range-nostem           True     402
60                 tfidf-trigram           True     396
54         tfidf-quintgram-range           True     394
48           tfidf-bigram-nostem           True     385
43                       jaccard           True     374
47                  tfidf-bigram           True     369
49                  tfidf-nostem           True     365
46                         tfidf           True 

In [4]:
print("* Import citations for cases...")
citations = pd.read_csv('../inputdata/all_cases_citations.csv')
print(" Successfully imported citations!")

def find_cited_cases(celexnumber):
    global citations
    relevantsource = citations[citations['source'] == celexnumber]
    return relevantsource['target'].tolist()

def exists_citation_link_between(celexnumber1,celexnumber2):
    global citations
    relevantsource1 = citations[citations['source'] == celexnumber1]
    relevantsource2 = citations[citations['source'] == celexnumber2]
    if celexnumber2 in relevantsource1['target'].tolist() or celexnumber1 in relevantsource2['target'].tolist():
        return True
    return False

* Import citations for cases...
 Successfully imported citations!


In [5]:
resultsdf = pd.read_csv("../outputdata/results.csv")

unique_samples_ph = resultsdf[resultsdf['source_case_topic'] == 'public health']['source_case'].unique()
unique_samples_sp = resultsdf[resultsdf['source_case_topic'] == 'social policy']['source_case'].unique()
unique_samples_dp = resultsdf[resultsdf['source_case_topic'] == 'data protection']['source_case'].unique()

print("no. of PH sample cases:",len(unique_samples_ph))
print("no. of SP sample cases:",len(unique_samples_sp))
print("no. of DP sample cases:",len(unique_samples_dp))

no. of PH sample cases: 63
no. of SP sample cases: 85
no. of DP sample cases: 29


In [6]:
cited_cases_ph = []
cited_cases_sp = []
cited_cases_dp = []
cited_cases = []

for item in unique_samples_ph:
    cited_cases_ph.extend(find_cited_cases(item))
    
for item in unique_samples_sp:
    cited_cases_sp.extend(find_cited_cases(item))
    
for item in unique_samples_dp:
    cited_cases_dp.extend(find_cited_cases(item))

cited_cases.extend(cited_cases_ph)
cited_cases.extend(cited_cases_sp)
cited_cases.extend(cited_cases_dp)

print("total no. of (non)unique citations for Public Health cases:",len(cited_cases_ph))
print("total no. of (non)unique citations for Social Policy cases:",len(cited_cases_sp))
print("total no. of (non)unique citations for Data Protection cases:",len(cited_cases_dp))
print("total no. of (non)unique citations for ALL cases:",(len(cited_cases_ph) + len(cited_cases_sp) + len(cited_cases_dp)))


total no. of (non)unique citations for Public Health cases: 176
total no. of (non)unique citations for Social Policy cases: 308
total no. of (non)unique citations for Data Protection cases: 138
total no. of (non)unique citations for ALL cases: 622


In [7]:
unique_cited_cases_ph = overlapsDf[overlapsDf['similar_case'].isin(cited_cases_ph)]
unique_cited_cases_ph = unique_cited_cases_ph[unique_cited_cases_ph['source_case_topic'] == 'public health']
#print("citation overlap for public health sample cases:",len(unique_cited_cases_ph))

unique_cited_cases_sp = overlapsDf[overlapsDf['similar_case'].isin(cited_cases_sp)]
unique_cited_cases_sp = unique_cited_cases_sp[unique_cited_cases_sp['source_case_topic'] == 'social policy']
#print("citation overlap for social policy sample cases:",len(unique_cited_cases_sp))

unique_cited_cases_dp = overlapsDf[overlapsDf['similar_case'].isin(cited_cases_dp)]
unique_cited_cases_dp = unique_cited_cases_dp[unique_cited_cases_dp['source_case_topic'] == 'data protection']
#print("citation overlap for data protection sample cases:",len(unique_cited_cases_dp))
#print("total overlap:",len(unique_cited_cases_ph) + len(unique_cited_cases_sp) + len(unique_cited_cases_dp))

#print()
unique_cited_cases_ph = unique_cited_cases_ph.groupby(['method']).similar_case.agg('count').to_frame('counts').reset_index()
unique_cited_cases_ph= unique_cited_cases_ph.sort_values(by ='counts',ascending=False)
#print(unique_cited_cases_ph.head(100))

#print()
unique_cited_cases_sp = unique_cited_cases_sp.groupby(['method']).similar_case.agg('count').to_frame('counts').reset_index()
unique_cited_cases_sp= unique_cited_cases_sp.sort_values(by ='counts',ascending=False)
#print(unique_cited_cases_sp.head(100))

#print()
unique_cited_cases_dp = unique_cited_cases_dp.groupby(['method']).similar_case.agg('count').to_frame('counts').reset_index()
unique_cited_cases_dp = unique_cited_cases_dp.sort_values(by ='counts',ascending=False)
#print(unique_cited_cases_dp.head(100))
#print(len(unique_cited_cases_dp))

appended1 = unique_cited_cases_ph.append(unique_cited_cases_sp)
finalappended = appended1.append(unique_cited_cases_dp)
finalappended = finalappended.groupby(['method']).sum()
finalappended = finalappended.sort_values(by ='counts',ascending=False)

print(finalappended.head(200))


                              counts
method                              
tfidf-quintgram                  273
tfidf-quintgram-nostem           270
tfidf-sesgram-nostem             269
tfidf-sesgram                    268
tfidf-quadgram-nostem            267
tfidf-quadgram                   265
tfidf-quintgram-range-nostem     263
tfidf-sepgram-nostem             262
tfidf-trigram-nostem             260
tfidf-sepgram                    257
tfidf-quintgram-range            256
tfidf-trigram                    256
jaccard                          247
tfidf-bigram-nostem              243
tfidf-bigram                     239
tfidf-nostem                     236
tfidf                            225
jaccard-stem                     220
GoogleNews-300-WMD               106
Law2Vec-200-WMD                  106
WMD_256_e30                       73
doc2vec_256_e30                   31
doc2vec_256_e50                   27
doc2vec_256                       27
doc2vec_256_e40                   26
d

In [8]:
mainmethods = ['tfidf-quintgram-nostem','jaccard','tfidf-nostem','GoogleNews-300-WMD','doc2vec_256_e30','law2vec_200','GoogleNews_300','WMD_256_e30','Law2Vec-200-WMD']

#print(finalappended.columns)
#finalappended.columns = ['method','counts']
finalappended['method'] = finalappended.index
#finalappended.reset_index(level=0, inplace=True)
finalappended = finalappended[finalappended['method'].isin(mainmethods)]
                              
print(finalappended.head(100))


                        counts                  method
method                                                
tfidf-quintgram-nostem     270  tfidf-quintgram-nostem
jaccard                    247                 jaccard
tfidf-nostem               236            tfidf-nostem
GoogleNews-300-WMD         106      GoogleNews-300-WMD
Law2Vec-200-WMD            106         Law2Vec-200-WMD
WMD_256_e30                 73             WMD_256_e30
doc2vec_256_e30             31         doc2vec_256_e30
law2vec_200                 22             law2vec_200
GoogleNews_300              19          GoogleNews_300
